In [1]:
import os
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss, roc_auc_score, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import warnings

plt.rc('font', family='Malgun Gothic')
plt.rcParams['axes.unicode_minus'] = False
warnings.filterwarnings('ignore')

BASE_DIR = os.path.join('..')
DATA_DIR = os.path.join(BASE_DIR, 'data_final')

REG_PATH = os.path.join(DATA_DIR, 'regression_dataset_final.csv')
MET_PATH = os.path.join(DATA_DIR, 'performance_metrics_final.csv')
TUNE_PATH = os.path.join(DATA_DIR, 'tuning_extended_final.csv')

print("데이터 로드")
df_reg = pd.read_csv(REG_PATH)
df_met = pd.read_csv(MET_PATH)
df_tune = pd.read_csv(TUNE_PATH)

print(f"회귀 데이터셋 크기: {len(df_reg)}")
print(f"성능 메트릭 크기: {len(df_met)}")

데이터 로드
회귀 데이터셋 크기: 33016
성능 메트릭 크기: 2908


In [2]:
print("\n[다중공선성 진단]")
full_feature_cols = ['doc_length', 'query_length', 'query_avg_token_len',
                     'query_unique_ratio', 'query_match_count', 'query_match_ratio',
                     'dominant_topic', 'dominant_prob']

X_vif = df_reg[df_reg['model'] == 'BIM'][full_feature_cols].copy()
X_vif = X_vif.dropna()

vif_data = pd.DataFrame()
vif_data["Feature"] = full_feature_cols
vif_data["VIF"] = [variance_inflation_factor(X_vif.values, i) for i in range(len(full_feature_cols))]
vif_data = vif_data.sort_values('VIF', ascending=False)

print(vif_data.to_string(index=False))
print("\nVIF > 10이면 심각한 다중공선성")


[다중공선성 진단]
            Feature        VIF
 query_unique_ratio 101.640988
  query_match_count  85.764727
       query_length  81.861196
  query_match_ratio  62.290909
query_avg_token_len  25.764601
      dominant_prob  13.945969
     dominant_topic   3.478408
         doc_length   2.990859

VIF > 10이면 심각한 다중공선성


In [3]:
print("\n[변수 스케일링]")
df_reg['doc_length_k'] = df_reg['doc_length'] / 1000
print("doc_length를 1000으로 나눔 (해석: 1000자 단위)")

feature_cols = ['doc_length_k', 'query_length', 'dominant_topic']


[변수 스케일링]
doc_length를 1000으로 나눔 (해석: 1000자 단위)


In [4]:
print("\n[최종 사용 변수 VIF 확인]")
X_final = df_reg[df_reg['model'] == 'BIM'][feature_cols].copy()
X_final = X_final.dropna()

vif_final = pd.DataFrame()
vif_final["Feature"] = feature_cols
vif_final["VIF"] = [variance_inflation_factor(X_final.values, i) for i in range(len(feature_cols))]
print(vif_final.to_string(index=False))

print("\n[변수 설명]")
print("y = a1*doc_length_k + a2*query_length + a3*dominant_topic + b")
print("  a1 (doc_length_k):    문서 길이 (1000자 단위)")
print("  a2 (query_length):    쿼리 길이 (형태소 개수)")
print("  a3 (dominant_topic):  도메인 (0~9)")

print("\n[스케일링 후 통계량]")
print(df_reg[feature_cols + ['relevance']].describe())

def run_logit_weighted(model_name, subset_df, w_doc=1.0, w_query=1.0, w_topic=1.0):
    X = subset_df[['doc_length_k', 'query_length']].copy()
    X['doc_length_k'] *= w_doc
    X['query_length'] *= w_query

    topics = pd.get_dummies(subset_df['dominant_topic'], prefix='topic', drop_first=True)

    topics = topics * w_topic

    X = pd.concat([X, topics], axis=1)

    y = subset_df['relevance']
    X = sm.add_constant(X)

    try:
        model = sm.Logit(y, X)
        result = model.fit(disp=0, maxiter=100)

        df_res = pd.DataFrame({
            'Model': model_name,
            'Feature': result.params.index,
            'Coefficient': result.params.values,
            'Odds_Ratio': np.exp(result.params.values),
            'P_value': result.pvalues.values,
            'Lower_CI': np.exp(result.conf_int()[0].values),
            'Upper_CI': np.exp(result.conf_int()[1].values)
        })

        pseudo_r2 = result.prsquared
        aic = result.aic

        return df_res, pseudo_r2, aic
    except Exception as e:
        print(f"오류 발생 in {model_name}: {e}")
        return pd.DataFrame(), 0, 0


[최종 사용 변수 VIF 확인]
       Feature      VIF
  doc_length_k 2.065526
  query_length 2.858230
dominant_topic 2.252184

[변수 설명]
y = a1*doc_length_k + a2*query_length + a3*dominant_topic + b
  a1 (doc_length_k):    문서 길이 (1000자 단위)
  a2 (query_length):    쿼리 길이 (형태소 개수)
  a3 (dominant_topic):  도메인 (0~9)

[스케일링 후 통계량]
       doc_length_k  query_length  dominant_topic     relevance
count  33016.000000  33016.000000    33016.000000  33016.000000
mean      17.010184     13.781500        3.820935      0.307730
std       19.579956      7.231487        2.397266      0.461561
min        0.272000      1.000000        0.000000      0.000000
25%        2.856250      8.000000        2.000000      0.000000
50%        9.887000     13.000000        3.000000      0.000000
75%       24.092000     18.000000        5.000000      1.000000
max      102.419000     39.000000        9.000000      1.000000


In [5]:
print("\n[1단계: 기본 회귀 분석 (가중치 1.0, 1.0, 1.0)]")
res_bim, r2_bim, aic_bim = run_logit_weighted("BIM", df_reg[df_reg['model'] == 'BIM'])
res_bm25, r2_bm25, aic_bm25 = run_logit_weighted("BM25", df_reg[df_reg['model'] == 'BM25_Best'])

print(f"BIM - Pseudo R2: {r2_bim:.4f}, AIC: {aic_bim:.2f}")
print(f"BM25 - Pseudo R2: {r2_bm25:.4f}, AIC: {aic_bm25:.2f}")

baseline_results = pd.concat([res_bim, res_bm25], ignore_index=True)
baseline_results = baseline_results[baseline_results['Feature'] != 'const']

print("\n[주요 변수 Odds Ratio]")
for feature in ['doc_length_k', 'query_length', 'dominant_topic']:
    print(f"\n{feature}:")
    for model in ['BIM', 'BM25']:
        row = baseline_results[(baseline_results['Model'] == model) &
                               (baseline_results['Feature'] == feature)]
        if len(row) > 0:
            or_val = row['Odds_Ratio'].values[0]
            p_val = row['P_value'].values[0]
            ci_low = row['Lower_CI'].values[0]
            ci_high = row['Upper_CI'].values[0]

            if 'doc_length' in feature:
                interpret = f"(1000자 증가 시 성공률 {(or_val-1)*100:+.2f}%)"
            elif 'query' in feature:
                interpret = f"(1형태소 증가 시 성공률 {(or_val-1)*100:+.2f}%)"
            else:
                interpret = f"(1단위 증가 시 성공률 {(or_val-1)*100:+.2f}%)"

            print(f"  {model}: OR={or_val:.6f} {interpret}")
            print(f"        95% CI: {ci_low:.4f}-{ci_high:.4f}, p={p_val:.4f}")


[1단계: 기본 회귀 분석 (가중치 1.0, 1.0, 1.0)]
BIM - Pseudo R2: 0.4240, AIC: 12066.22
BM25 - Pseudo R2: 0.0145, AIC: 19560.61

[주요 변수 Odds Ratio]

doc_length_k:
  BIM: OR=0.848458 (1000자 증가 시 성공률 -15.15%)
        95% CI: 0.8438-0.8531, p=0.0000
  BM25: OR=1.002023 (1000자 증가 시 성공률 +0.20%)
        95% CI: 0.9983-1.0057, p=0.2825

query_length:
  BIM: OR=1.091478 (1형태소 증가 시 성공률 +9.15%)
        95% CI: 1.0843-1.0987, p=0.0000
  BM25: OR=1.031351 (1형태소 증가 시 성공률 +3.14%)
        95% CI: 1.0265-1.0362, p=0.0000

dominant_topic:


In [8]:
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import MinMaxScaler
import itertools

print("\n[2단계: 수동 가중치 튜닝 실험 - Manual Linear Scoring]")
print("Score = w_doc * doc_length_k + w_query * query_length + w_topic * dominant_prob")

scaler = MinMaxScaler()

weight_candidates_doc = [0.0, 0.5, 1.0, 1.5, 2.0, 3.0, -0.5, -1.0]
weight_candidates_query = [0.0, 0.5, 1.0, 1.5, 2.0, 3.0]
weight_candidates_topic = [0.0, 0.5, 1.0, 1.5, 2.0]

print(f"가중치 후보:")
print(f"  w_doc: {weight_candidates_doc}")
print(f"  w_query: {weight_candidates_query}")
print(f"  w_topic: {weight_candidates_topic}")
print(f"총 조합: {len(weight_candidates_doc) * len(weight_candidates_query) * len(weight_candidates_topic)}개")

results = []

# 변수 목록 (ID 대신 확률 사용)
manual_features = ['doc_length_k', 'query_length', 'dominant_prob']

for model_type in ['BIM', 'BM25_Best']:
    print(f"\n{model_type} 가중치 탐색 중")

    subset = df_reg[df_reg['model'] == model_type].copy()
    y_true = subset['relevance']

    # 스케일링
    X_scaled = pd.DataFrame(
        scaler.fit_transform(subset[manual_features]),
        columns=manual_features,
        index=subset.index
    )

    for w_doc, w_query, w_topic in tqdm(
            itertools.product(weight_candidates_doc, weight_candidates_query, weight_candidates_topic),
            total=len(weight_candidates_doc) * len(weight_candidates_query) * len(weight_candidates_topic),
            desc=f"{model_type}"
    ):
        score = (w_doc * X_scaled['doc_length_k']) + \
                (w_query * X_scaled['query_length']) + \
                (w_topic * X_scaled['dominant_prob'])

        try:
            auc = roc_auc_score(y_true, score)
        except ValueError:
            auc = 0.5

        results.append({
            'model': model_type,
            'w_doc': w_doc,
            'w_query': w_query,
            'w_topic': w_topic,
            'AUC': auc
        })

df_weight_tuning = pd.DataFrame(results)

print("\n[BIM 가중치 튜닝 결과 - 상위 20개]")
bim_results = df_weight_tuning[df_weight_tuning['model']=='BIM'].sort_values('AUC', ascending=False)
print(bim_results.head(20).to_string(index=False))

print("\n[BM25 가중치 튜닝 결과 - 상위 20개]")
bm25_results = df_weight_tuning[df_weight_tuning['model']=='BM25_Best'].sort_values('AUC', ascending=False)
print(bm25_results.head(20).to_string(index=False))

best_config_bim = bim_results.iloc[0]
best_config_bm25 = bm25_results.iloc[0]

print(f"\n[BIM 최적 가중치]")
print(f"  w_doc={best_config_bim['w_doc']}, w_query={best_config_bim['w_query']}, w_topic={best_config_bim['w_topic']}")
print(f"  AUC={best_config_bim['AUC']:.4f}")

print(f"\n[BM25 최적 가중치]")
print(f"  w_doc={best_config_bm25['w_doc']}, w_query={best_config_bm25['w_query']}, w_topic={best_config_bm25['w_topic']}")
print(f"  AUC={best_config_bm25['AUC']:.4f}")

print("\n[BIM vs BM25 비교]")
print(f"  BIM 최적 w_doc: {best_config_bim['w_doc']}")
print(f"  BM25 최적 w_doc: {best_config_bm25['w_doc']}")

df_weight_tuning.to_csv(os.path.join(DATA_DIR, 'manual_tuning_results_final.csv'), index=False)


[2단계: 수동 가중치 튜닝 실험 - Manual Linear Scoring]
Score = w_doc * doc_length_k + w_query * query_length + w_topic * dominant_prob
가중치 후보:
  w_doc: [0.0, 0.5, 1.0, 1.5, 2.0, 3.0, -0.5, -1.0]
  w_query: [0.0, 0.5, 1.0, 1.5, 2.0, 3.0]
  w_topic: [0.0, 0.5, 1.0, 1.5, 2.0]
총 조합: 240개

BIM 가중치 탐색 중


BIM: 100%|██████████| 240/240 [00:00<00:00, 332.70it/s]



BM25_Best 가중치 탐색 중


BM25_Best: 100%|██████████| 240/240 [00:00<00:00, 327.51it/s]


[BIM 가중치 튜닝 결과 - 상위 20개]
model  w_doc  w_query  w_topic      AUC
  BIM   -0.5      0.0      0.0 0.901143
  BIM   -1.0      0.0      0.0 0.901143
  BIM   -1.0      0.5      0.0 0.886659
  BIM   -1.0      0.5      0.5 0.827368
  BIM   -1.0      1.0      0.0 0.821125
  BIM   -0.5      0.5      0.0 0.821125
  BIM   -1.0      0.0      0.5 0.819755
  BIM   -1.0      1.0      0.5 0.792871
  BIM   -1.0      1.5      0.0 0.768337
  BIM   -1.0      1.5      0.5 0.753500
  BIM   -1.0      0.5      1.0 0.738963
  BIM   -0.5      0.5      0.5 0.731563
  BIM   -1.0      1.0      1.0 0.731563
  BIM   -1.0      2.0      0.0 0.730341
  BIM   -0.5      1.0      0.0 0.730341
  BIM   -0.5      0.0      0.5 0.728424
  BIM   -1.0      0.0      1.0 0.728424
  BIM   -1.0      2.0      0.5 0.721402
  BIM   -1.0      1.5      1.0 0.715280
  BIM   -1.0      2.0      1.0 0.697332

[BM25 가중치 튜닝 결과 - 상위 20개]
    model  w_doc  w_query  w_topic      AUC
BM25_Best    0.5      1.5      0.0 0.573167
BM25_Best    1.0   

In [9]:
print("\n" + "="*60)
print("[최적 가중치 분석]")
print("="*60)

print(f"\n[1] BIM 모델 분석")
print(f"  - 최적 설정: w_doc={best_config_bim['w_doc']}, AUC={best_config_bim['AUC']:.4f}")
print(f"  - 해석: AUC가 0.90으로 매우 높고 w_doc가 음수(-0.5)인 것은,")
print(f"         BIM이 '짧은 문서'에 극도로 편향되어 있음을 의미합니다.")
print(f"         즉, 문서 길이($x_1$)가 검색 성능을 좌지우지하는 불안정한 모델입니다.")

print(f"\n[2] BM25 모델 분석")
print(f"  - 최적 설정: w_doc={best_config_bm25['w_doc']}, w_query={best_config_bm25['w_query']}, AUC={best_config_bm25['AUC']:.4f}")
print(f"  - 해석: AUC가 0.57로 낮게 나온 것은, 문서 길이나 쿼리 길이 같은 단순 변수로는")
print(f"         성공 여부를 예측하기 어렵다는 뜻입니다. (즉, 특정 조건에 편향되지 않음)")
print(f"         특히 w_query(1.5)가 w_doc(0.5)보다 큰 것은, 문서 길이보다는")
print(f"         '질문의 구체성'이 성능에 더 긍정적인 영향을 미친다는 바람직한 신호입니다.")

print(f"\n[3] 종합 비교 (BIM vs BM25)")
if best_config_bim['AUC'] > best_config_bm25['AUC']:
    print(f"  - 결론: BIM의 높은 AUC는 '성능이 좋다'는 뜻이 아니라 '편향이 심하다'는 뜻입니다.")
    print(f"         반면 BM25는 낮은 AUC를 통해 '길이 정규화($b$)가 성공적으로 작동했음'을 증명했습니다.")
    print(f"         따라서 BM25가 더 공정하고 신뢰할 수 있는 모델입니다.")

print("="*60)


[최적 가중치 분석]

[1] BIM 모델 분석
  - 최적 설정: w_doc=-0.5, AUC=0.9011
  - 해석: AUC가 0.90으로 매우 높고 w_doc가 음수(-0.5)인 것은,
         BIM이 '짧은 문서'에 극도로 편향되어 있음을 의미합니다.
         즉, 문서 길이($x_1$)가 검색 성능을 좌지우지하는 불안정한 모델입니다.

[2] BM25 모델 분석
  - 최적 설정: w_doc=0.5, w_query=1.5, AUC=0.5732
  - 해석: AUC가 0.57로 낮게 나온 것은, 문서 길이나 쿼리 길이 같은 단순 변수로는
         성공 여부를 예측하기 어렵다는 뜻입니다. (즉, 특정 조건에 편향되지 않음)
         특히 w_query(1.5)가 w_doc(0.5)보다 큰 것은, 문서 길이보다는
         '질문의 구체성'이 성능에 더 긍정적인 영향을 미친다는 바람직한 신호입니다.

[3] 종합 비교 (BIM vs BM25)
  - 결론: BIM의 높은 AUC는 '성능이 좋다'는 뜻이 아니라 '편향이 심하다'는 뜻입니다.
         반면 BM25는 낮은 AUC를 통해 '길이 정규화($b$)가 성공적으로 작동했음'을 증명했습니다.
         따라서 BM25가 더 공정하고 신뢰할 수 있는 모델입니다.


In [10]:
print("\n[3단계: 종합 성능 평가]")
map_bim = df_met[df_met['model'] == 'BIM']['AP'].mean()
map_bm25 = df_met[df_met['model'] == 'BM25_Best']['AP'].mean()

p10_bim = df_met[df_met['model'] == 'BIM']['P@10'].mean()
p10_bm25 = df_met[df_met['model'] == 'BM25_Best']['P@10'].mean()

r10_bim = df_met[df_met['model'] == 'BIM']['R@10'].mean()
r10_bm25 = df_met[df_met['model'] == 'BM25_Best']['R@10'].mean()

or_doc_bim = baseline_results[(baseline_results['Model'] == 'BIM') &
                              (baseline_results['Feature'] == 'doc_length_k')]['Odds_Ratio'].values[0]
or_doc_bm25 = baseline_results[(baseline_results['Model'] == 'BM25') &
                               (baseline_results['Feature'] == 'doc_length_k')]['Odds_Ratio'].values[0]

or_query_bim = baseline_results[(baseline_results['Model'] == 'BIM') &
                                (baseline_results['Feature'] == 'query_length')]['Odds_Ratio'].values[0]
or_query_bm25 = baseline_results[(baseline_results['Model'] == 'BM25') &
                                 (baseline_results['Feature'] == 'query_length')]['Odds_Ratio'].values[0]

or_topic_bim = baseline_results[(baseline_results['Model'] == 'BIM') &
                                (baseline_results['Feature'] == 'dominant_topic')]['Odds_Ratio'].values[0]
or_topic_bm25 = baseline_results[(baseline_results['Model'] == 'BM25') &
                                 (baseline_results['Feature'] == 'dominant_topic')]['Odds_Ratio'].values[0]

comprehensive_results = pd.DataFrame({
    'Model': ['BIM', 'BM25_Best'],
    'MAP': [map_bim, map_bm25],
    'P@10': [p10_bim, p10_bm25],
    'R@10': [r10_bim, r10_bm25],
    'Pseudo_R2': [r2_bim, r2_bm25],
    'AIC': [aic_bim, aic_bm25],
    'best_AUC': [best_config_bim['auc'], best_config_bm25['auc']],
    'doc_length_OR': [or_doc_bim, or_doc_bm25],
    'query_length_OR': [or_query_bim, or_query_bm25],
    'dominant_topic_OR': [or_topic_bim, or_topic_bm25],
    'best_w_doc': [best_config_bim['w_doc'], best_config_bm25['w_doc']],
    'best_w_query': [best_config_bim['w_query'], best_config_bm25['w_query']],
    'best_w_topic': [best_config_bim['w_topic'], best_config_bm25['w_topic']],
    'best_config': [best_config_bim['config'], best_config_bm25['config']]
})

print("\n[종합 결과 테이블]")
print(comprehensive_results.to_string(index=False))


[3단계: 종합 성능 평가]


IndexError: index 0 is out of bounds for axis 0 with size 0

In [13]:
# ==============================================================================
# [3단계 수정] 모든 변수(문서/쿼리/각 토픽)의 Odds Ratio 상세 비교
# ==============================================================================
print("\n" + "="*60)
print("[상세 분석] 변수별 Odds Ratio 전수 조사 (BIM vs BM25)")
print("="*60)

# 1. Pivot Table 생성 (행: 변수명, 열: 모델명, 값: Odds Ratio)
# baseline_results는 Step 1에서 이미 생성된 상태여야 합니다.
detailed_or = baseline_results.pivot(index='Feature', columns='Model', values='Odds_Ratio')

# 2. 보기 좋게 정렬 (문서길이 -> 쿼리길이 -> 토픽0 -> 토픽1 ...)
# 토픽 변수들만 뽑아서 정렬
topic_vars = sorted([idx for idx in detailed_or.index if idx.startswith('topic_')])
# 최종 순서 지정
sort_order = ['doc_length_k', 'query_length'] + topic_vars

# 정렬 적용 (데이터에 없는 변수가 있을 경우 에러 방지 위해 intersection 사용)
final_index = [idx for idx in sort_order if idx in detailed_or.index]
detailed_or = detailed_or.reindex(final_index)

# 3. 해석을 위한 컬럼 추가 (BIM 대비 BM25가 얼마나 공정한가?)
# 1.0에 가까울수록 영향력이 중립적(공정)임
detailed_or['Interpretation'] = ''

for idx in detailed_or.index:
    bim_val = detailed_or.at[idx, 'BIM']
    bm25_val = detailed_or.at[idx, 'BM25']

    # 문서 길이에 대한 해석
    if idx == 'doc_length_k':
        if bim_val < 0.9:
            detailed_or.at[idx, 'Interpretation'] = 'BIM: 짧은 문서 편향 (심각)'
        if 0.95 < bm25_val < 1.05:
            detailed_or.at[idx, 'Interpretation'] += ' / BM25: 편향 제거 성공'

    # 토픽에 대한 해석 (BIM vs BM25 차이)
    elif idx.startswith('topic'):
        diff = abs(bim_val - bm25_val)
        if diff < 0.05:
            detailed_or.at[idx, 'Interpretation'] = '영향력 미미 (차이 없음)'
        else:
            detailed_or.at[idx, 'Interpretation'] = f'가중치 차이: {diff:.4f}'

# 4. 출력
pd.set_option('display.max_rows', None) # 모든 행 다 보기
print(detailed_or)
pd.reset_option('display.max_rows')

# -----------------------------------------------------------
# [추가] 성능 지표 요약 테이블 (MAP, AUC 등)
# -----------------------------------------------------------
print("\n[모델 성능 요약]")
summary_table = pd.DataFrame({
    'Metric': ['MAP', 'P@10', 'Best AUC (Manual Tuning)', 'Pseudo R2'],
    'BIM': [
        df_met[df_met['model']=='BIM']['AP'].mean(),
        df_met[df_met['model']=='BIM']['P@10'].mean(),
        best_config_bim['AUC'],  # Step 2 결과 변수
        r2_bim                   # Step 1 결과 변수
    ],
    'BM25': [
        df_met[df_met['model']=='BM25_Best']['AP'].mean(),
        df_met[df_met['model']=='BM25_Best']['P@10'].mean(),
        best_config_bm25['AUC'], # Step 2 결과 변수
        r2_bm25                  # Step 1 결과 변수
    ]
})
print(summary_table.to_string(index=False))

# -----------------------------------------------------------
# [CSV 저장] 이 테이블을 파일로 저장 (보고서 첨부용)
# -----------------------------------------------------------
# save_path = os.path.join(DATA_DIR, 'detailed_odds_ratios_final.csv')
# detailed_or.to_csv(save_path)
# print(f"\n상세 OR 테이블 저장 완료: {save_path}")


[상세 분석] 변수별 Odds Ratio 전수 조사 (BIM vs BM25)
Model              BIM      BM25                       Interpretation
Feature                                                              
doc_length_k  0.848458  1.002023  BIM: 짧은 문서 편향 (심각) / BM25: 편향 제거 성공
query_length  1.091478  1.031351                                     
topic_1       0.747916  0.610428                       가중치 차이: 0.1375
topic_2       0.877872  0.631907                       가중치 차이: 0.2460
topic_3       0.793264  0.847787                       가중치 차이: 0.0545
topic_4       1.013599  0.765924                       가중치 차이: 0.2477
topic_5       0.714491  0.593138                       가중치 차이: 0.1214
topic_6       1.073847  0.836040                       가중치 차이: 0.2378
topic_7       0.969990  0.687302                       가중치 차이: 0.2827
topic_8       0.909469  0.765449                       가중치 차이: 0.1440
topic_9       1.209238  0.845587                       가중치 차이: 0.3637

[모델 성능 요약]
                  Metric      BIM 

In [ ]:
print("\n[4단계: 결과 저장]")
baseline_results.to_csv(os.path.join(DATA_DIR, 'odds_ratio_results_final.csv'), index=False)
df_weight_tuning.to_csv(os.path.join(DATA_DIR, 'weight_tuning_results_final.csv'), index=False)
comprehensive_results.to_csv(os.path.join(DATA_DIR, 'comprehensive_results_final.csv'), index=False)
vif_data.to_csv(os.path.join(DATA_DIR, 'vif_analysis_final.csv'), index=False)

In [ ]:
print("\n[5단계: 시각화]")

def plot_forest_comparison(data, feature_name, title, xlabel):
    subset = data[data['Feature'] == feature_name]
    if len(subset) == 0:
        print(f"경고: {feature_name}에 대한 데이터 없음")
        return

    plt.figure(figsize=(10, 3))
    y_pos = range(len(subset))

    err = [subset['Odds_Ratio'] - subset['Lower_CI'], subset['Upper_CI'] - subset['Odds_Ratio']]

    colors = ['blue' if m == 'BIM' else 'red' for m in subset['Model']]

    plt.errorbar(subset['Odds_Ratio'], y_pos, xerr=err, fmt='o', capsize=5, color='black')
    plt.scatter(subset['Odds_Ratio'], y_pos, c=colors, s=100, zorder=3)

    plt.axvline(x=1.0, color='gray', linestyle='--', linewidth=1)

    plt.yticks(y_pos, subset['Model'])
    plt.xlabel(xlabel)
    plt.title(title, fontsize=14, fontweight='bold')

    for i, (_, row) in enumerate(subset.iterrows()):
        plt.text(row['Odds_Ratio'], i - 0.3, f"OR: {row['Odds_Ratio']:.4f}\n(p={row['P_value']:.3f})",
                 ha='center', va='top', fontsize=9)

    plt.tight_layout()
    plt.savefig(os.path.join(DATA_DIR, f'{feature_name}_forest_plot_final.png'), dpi=300)
    plt.show()

print("a1: 문서 길이 영향력 시각화")
plot_forest_comparison(baseline_results, 'doc_length_k',
                       'a1: 문서 길이 Odds Ratio (1000자 단위)',
                       'Odds Ratio (1.0보다 작으면 길이가 길수록 불리함)')

print("a2: 쿼리 길이 영향력 시각화")
plot_forest_comparison(baseline_results, 'query_length',
                       'a2: 쿼리 길이 Odds Ratio',
                       'Odds Ratio (1.0보다 크면 쿼리가 길수록 성공 확률 증가)')

print("a3: 도메인 토픽 영향력 시각화")
plot_forest_comparison(baseline_results, 'dominant_topic',
                       'a3: 도미넌트 토픽 Odds Ratio',
                       'Odds Ratio (영향력)')

plt.figure(figsize=(12, 6))
x = np.arange(len(comprehensive_results))
width = 0.35

plt.subplot(1, 2, 1)
plt.bar(x - width/2, comprehensive_results['MAP'], width, label='MAP', alpha=0.8)
plt.bar(x + width/2, comprehensive_results['P@10'], width, label='P@10', alpha=0.8)
plt.xlabel('모델')
plt.ylabel('점수')
plt.title('검색 성능 비교 (MAP vs P@10)')
plt.xticks(x, comprehensive_results['Model'])
plt.legend()
plt.grid(axis='y', linestyle=':', alpha=0.7)

plt.subplot(1, 2, 2)
plt.bar(x, comprehensive_results['Pseudo_R2'], alpha=0.8, color=['blue', 'red'])
plt.xlabel('모델')
plt.ylabel('Pseudo R-squared')
plt.title('회귀 모델 설명력')
plt.xticks(x, comprehensive_results['Model'])
plt.grid(axis='y', linestyle=':', alpha=0.7)

plt.tight_layout()
plt.savefig(os.path.join(DATA_DIR, 'performance_comparison_final.png'), dpi=300)
plt.show()

plt.figure(figsize=(12, 8))
pivot_weight = df_weight_tuning.pivot_table(
    index='config',
    columns='model',
    values='auc'
)
sns.heatmap(pivot_weight, annot=True, fmt=".4f", cmap="YlGnBu", linewidths=.5)
plt.title('가중치 설정별 AUC 히트맵')
plt.xlabel('모델')
plt.ylabel('가중치 설정')
plt.tight_layout()
plt.savefig(os.path.join(DATA_DIR, 'weight_tuning_heatmap_final.png'), dpi=300)
plt.show()

important_features = baseline_results[baseline_results['Feature'].isin(['doc_length_k', 'query_length', 'dominant_topic'])]
plt.figure(figsize=(10, 6))
sns.barplot(x='Feature', y='Odds_Ratio', hue='Model', data=important_features, palette=['#4c72b0', '#c44e52'])
plt.axhline(y=1.0, color='black', linestyle='--', linewidth=1)
plt.title('주요 변수별 Odds Ratio 비교 (a1, a2, a3)')
plt.xlabel('변수')
plt.ylabel('Odds Ratio')
plt.legend(title='모델')
plt.grid(axis='y', linestyle=':', alpha=0.7)
plt.tight_layout()
plt.savefig(os.path.join(DATA_DIR, 'main_features_comparison_final.png'), dpi=300)
plt.show()

print("\n작업 완료")
print(f"저장된 파일:")
print(f"  - odds_ratio_results_final.csv")
print(f"  - weight_tuning_results_final.csv")
print(f"  - comprehensive_results_final.csv")
print(f"  - vif_analysis_final.csv")
print(f"  - 시각화 PNG 파일 4개")